In [ ]:

import ee

from fire_module_2 import step2
from fire_module import step1
ee.Initialize()


The fire scripts are broken down into 3 steps:
1. Prepare and export baseline spatial and temporal mean and standard deviation
    - This is specific to the year and land cover type
2. Prepare and export NBR anomalies 
3. Export Annual burn map by land cover

<img src="imgs/pipeline.png" alt="example of workflow as a pipeline" width="200"/>

step1(analysisYear, geometry, cover, covername)

Class for initializing the first steps in the burn anomalies mapping process and key input into the second step for generating burn maps.

Args:
- paramtersIO (object): Parameters for cloud and shadow masking and dictionary of cover names and export paths.
- analysisYear (int): The year to generate burn products.
- geometry (ee.FeatureCollection): A feature collection of at least 1 geometry for the ROI. If there are multiple features in a collection the geometry is found with the .geometry() method.
- cover (ee.Image): The land cover image that is described in paramterIO cover dictionary.
- coverName (str): The land cover name to generate burn products for.
        

In [ ]:
# Note: use test geom if wanted to run test exports it much smaller (and thus faster)
test_geom = ee.FeatureCollection(
    "projects/sig-misc-ee/assets/drc_fire/test_areas/test_area")
DRC_border = ee.FeatureCollection(
    "projects/ee-karistenneson/assets/BurnedBiomass/DRC_Training/DRC_Border")
ROC_border = ee.FeatureCollection(
    "projects/central-africa-silvacarbon/assets/roc_fire/tables/roc_geom")
# cover for drc
drc_16 = ee.Image('projects/sig-ee/FIRE/DRC/DIAF_2000forest')
# covers for roc
roc_10 = ee.Image("projects/sig-misc-ee/assets/roc_fire/landcover/roc_forest_cover_map_2010_gaf_fin01")

# drc cover name options:
# Forests without dry forest
# Dry forest or open forest
# roc cover name options:
# Forest
cover = roc_10
covername = "Forest"
year = 2010
region = ROC_border
fire = step1(year, region, cover, covername)

The preparing baseline step will take the longest to export. Test runs for DRC for 2016 typically finish ~12-19h.

prepare_script1() returns an imagecollection of the baseline mean, and stddevs 

export_image_collection takes the collection to export, and a function for how to export a specific collection in this case since we are making the baseline we pass in the export_baseline_landcover function.

optionally: you can set test = True to export a single image, and change the export scale exportScale=Int
export_image_collection(image_collection, export function, test: bool = False, exportScale: int = 30 )

In [ ]:
prep_baseline = fire.prepare_script1()

fire.export_image_collection(
    prep_baseline, fire.export_baseline_landcover)#test=True

Next the anomalies are created for each analysis period for our analysis year. Test runs for DRC for 2016 typically finish in ~9-15h.

If you neeeded to restart the notebook that is ok, but make sure the fire object has be initiated for the anomalies that need to be processed.

Likewise with the previous step test and exportScale can be adjusted

In [ ]:
anomaly_col = fire.script1()

fire.export_image_collection(
    anomaly_col, fire.export_nbr_anomalies)

Lastly the burn product cna be created for the analysis year using the 2nd step.

```
step2(step1, alpha, pVal):
Generate yearly burn product from NBR anomalies that incorporates MODIS hotspots.

Args:
    step1 (object): The initialized object used for earlier steps.
    alpha (float): p-value threshold for identifying fires.
    pVal (str): A string for the p-value image to use for thresholding. Can be pval_spatial or pval_temporal
```

In [ ]:
alpha = 0.05
pVal = 'pval_spatial'
fire2 = step2()
out = fire2.main(alpha, pVal, year, cover)
fire2.export_burn_yearly(
    out, region)